In [3]:
RANDOM_SEED = 42
SAMPLE_DATA_PERCENTAGE = 1.0
MIN_IOU_REQUIRED = 0.7  # The minimum IOU required to be a good detection (ie higher is more strict)
NUM_PROPOSALS = 300
CHECKPOINT_DIR = './idd_car_drop_checkpoints'

%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import torch as t
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
import numpy as np
import pickle
import h5py
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
from utils.config import opt
from utils.idd_utils import get_annotations
from model.utils.bbox_tools import bbox_iou

from data.dataset import Dataset, TestDataset, KittiDataset
from torch.utils.data import DataLoader
from utils.vis_tool import vis_bbox, vis_image
from utils.idd_utils import IndiaDrivingDataset
from ood_metrics import auroc, plot_roc, calc_metrics
from sklearn import svm
import pickle
import itertools
import torch
from model.faster_rcnn import nograd
from data.dataset import preprocess
from utils.mahalanobis import Mahal_Dist


import warnings
warnings.filterwarnings(action='once')

np.random.seed(RANDOM_SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
opt.voc_data_dir = "/media/tadenoud/DATADisk/datasets/kitti_2d/VOC2012/"
idd_data_dir = "/media/tadenoud/DATADisk/datasets/idd_detection/IDD_Detection/"

kitti_train_dataset = KittiDataset(opt, split='train')
kitti_val_dataset = KittiDataset(opt, split='val')
idd_val_dataset = IndiaDrivingDataset(idd_data_dir, split='val')

kitti_train_samples = round(len(kitti_train_dataset) * SAMPLE_DATA_PERCENTAGE)
kitti_val_samples = round(len(kitti_val_dataset) * SAMPLE_DATA_PERCENTAGE)
idd_val_samples = round(len(idd_val_dataset) * SAMPLE_DATA_PERCENTAGE)

KITTI_TRAIN_TO_USE = np.random.choice(kitti_train_samples, kitti_train_samples, replace=False)
KITTI_VAL_TO_USE = np.random.choice(kitti_val_samples, kitti_val_samples, replace=False)
IDD_VAL_TO_USE = np.random.choice(idd_val_samples, idd_val_samples, replace=False)

# Datasets after sampling a portion of them
kitti_train_dataset = torch.utils.data.Subset(kitti_train_dataset, KITTI_TRAIN_TO_USE)
kitti_val_dataset = torch.utils.data.Subset(kitti_val_dataset, KITTI_VAL_TO_USE)
idd_val_dataset = torch.utils.data.Subset(idd_val_dataset, IDD_VAL_TO_USE)

In [86]:
IndiaDrivingDataset??

# Reformat Kitti Val keys

In [63]:
with open('/media/tadenoud/DATADisk/datasets/kitti_2d/VOC2012/ImageSets/Main/val.txt', 'r') as f:
    kitti_val_files = [a.strip() for a in f.readlines()]

In [66]:
from shutil import copyfile


# Create the key to image map
with h5py.File(os.path.join(CHECKPOINT_DIR, 'kitti_cars_detections_val.hdf5'), 'r') as f:
    map_kitti_val_keys = dict((k, kitti_val_files[int(k)]) for k in f.keys())
    print(f['0'].keys())
    
copyfile(
    os.path.join(CHECKPOINT_DIR, 'kitti_cars_detections_val.hdf5'), 
    os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_val.hdf5')
)

# Change the keys in the vahdat file so they correspond to the image number
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_val.hdf5'), 'r+') as f:
    for k in f.keys():
        f[str(map_kitti_val_keys[k])] = f[k]
        del f[k]

<KeysViewHDF5 ['pred_boxes', 'pred_features_0', 'pred_features_1', 'pred_features_2', 'pred_features_3', 'pred_features_4', 'pred_label_correct', 'pred_labels', 'pred_max_ious', 'pred_num_detections', 'pred_scores']>


In [67]:
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_val.hdf5'), 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['000001', '000002', '000004', '000005', '000006', '000008', '000015', '000019', '000020', '000021', '000023', '000024', '000025', '000027', '000031', '000033', '000035', '000037', '000039', '000040', '000042', '000047', '000048', '000050', '000052', '000053', '000059', '000061', '000062', '000063', '000066', '000076', '000077', '000078', '000081', '000089', '000090', '000093', '000094', '000098', '000102', '000104', '000106', '000107', '000108', '000116', '000117', '000122', '000124', '000126', '000128', '000132', '000134', '000135', '000137', '000139', '000140', '000143', '000147', '000151', '000152', '000153', '000159', '000161', '000167', '000168', '000169', '000170', '000173', '000174', '000175', '000181', '000182', '000183', '000186', '000187', '000188', '000190', '000191', '000192', '000194', '000195', '000196', '000197', '000201', '000203', '000204', '000207', '000211', '000212', '000213', '000216', '000218', '000223', '000224', '000226', '000229', '000230', '0002

# Reformat Kitti Train keys

In [68]:
with open('/media/tadenoud/DATADisk/datasets/kitti_2d/VOC2012/ImageSets/Main/train.txt', 'r') as f:
    kitti_train_files = [a.strip() for a in f.readlines()]

In [69]:
from shutil import copyfile


# Create the key to image map
with h5py.File(os.path.join(CHECKPOINT_DIR, 'kitti_cars_detections_train.hdf5'), 'r') as f:
    map_kitti_train_keys = dict((k, kitti_train_files[int(k)]) for k in f.keys())
    print(f['0'].keys())
    
copyfile(
    os.path.join(CHECKPOINT_DIR, 'kitti_cars_detections_train.hdf5'), 
    os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_train.hdf5')
)

# Change the keys in the vahdat file so they correspond to the image number
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_train.hdf5'), 'r+') as f:
    for k in f.keys():
        f[str(map_kitti_train_keys[k])] = f[k]
        del f[k]

<KeysViewHDF5 ['pred_boxes', 'pred_features_0', 'pred_features_1', 'pred_features_2', 'pred_features_3', 'pred_features_4', 'pred_label_correct', 'pred_labels', 'pred_max_ious', 'pred_num_detections', 'pred_scores']>


In [70]:
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_kitti_cars_detections_train.hdf5'), 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['000003', '000007', '000009', '000010', '000011', '000012', '000013', '000014', '000016', '000017', '000018', '000022', '000026', '000029', '000030', '000032', '000034', '000036', '000038', '000041', '000043', '000044', '000045', '000046', '000049', '000051', '000055', '000056', '000057', '000060', '000064', '000067', '000068', '000069', '000071', '000072', '000073', '000074', '000075', '000079', '000080', '000082', '000083', '000084', '000085', '000086', '000087', '000088', '000091', '000092', '000095', '000096', '000097', '000099', '000100', '000101', '000105', '000110', '000111', '000112', '000113', '000114', '000115', '000119', '000120', '000121', '000123', '000125', '000127', '000129', '000131', '000133', '000136', '000138', '000141', '000142', '000144', '000145', '000146', '000148', '000149', '000150', '000154', '000155', '000157', '000158', '000160', '000162', '000163', '000164', '000165', '000166', '000171', '000172', '000176', '000177', '000178', '000179', '0001

# Reformat IDD val keys

In [72]:
with open('/media/tadenoud/DATADisk/datasets/idd_detection/IDD_Detection/val.txt', 'r') as f:
    idd_val_files = [a.strip() for a in f.readlines()]

In [76]:
map_idd_val_keys

{'0': 'frontFar/BLR-2018-04-16_16-14-27_frontFar/0006780',
 '1': 'frontFar/BLR-2018-04-16_16-14-27_frontFar/0001500',
 '10': 'frontFar/BLR-2018-04-16_16-14-27_frontFar/0010920',
 '100': 'frontFar/BLR-2018-04-16_16-14-27_frontFar/000258_r',
 '1000': 'frontFar/BLR-2018-04-19_17-16-55_frontFar/001674_r',
 '10000': 'highquality_16k/HYD-2018-08-24_13-22-50/0007776',
 '10001': 'highquality_16k/HYD-2018-08-24_13-22-50/0002418',
 '10002': 'highquality_16k/HYD-2018-08-24_13-22-50/0004204',
 '10003': 'highquality_16k/HYD-2018-08-24_13-32-50/0008664',
 '10004': 'highquality_16k/HYD-2018-08-24_13-32-50/0007866',
 '10005': 'highquality_16k/HYD-2018-08-24_13-32-50/0005130',
 '10006': 'highquality_16k/HYD-2018-08-24_13-32-50/0002812',
 '10007': 'highquality_16k/HYD-2018-08-24_13-32-50/0005434',
 '10008': 'highquality_16k/HYD-2018-08-24_13-32-50/0006308',
 '10009': 'highquality_16k/HYD-2018-08-24_13-32-50/0000684',
 '1001': 'frontFar/BLR-2018-04-19_17-16-55_frontFar/000042_r',
 '10010': 'highquality_1

In [83]:
from shutil import copyfile


# Create the key to image map
with h5py.File(os.path.join(CHECKPOINT_DIR, 'idd_cars_detections_val.hdf5'), 'r') as f:
    map_idd_val_keys = dict((k, idd_val_files[int(k)]) for k in f.keys())
    print(f['0'].keys())
    
copyfile(
    os.path.join(CHECKPOINT_DIR, 'idd_cars_detections_val.hdf5'), 
    os.path.join(CHECKPOINT_DIR, 'vahdat_idd_cars_detections_val.hdf5')
)

# Change the keys in the vahdat file so they correspond to the image number
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_idd_cars_detections_val.hdf5'), 'r+') as f:
    for k in f.keys():
        f[str(map_idd_val_keys[k]).replace('/', '|')] = f[k]
        del f[k]

<KeysViewHDF5 ['pred_boxes', 'pred_features_0', 'pred_features_1', 'pred_features_2', 'pred_features_3', 'pred_features_4', 'pred_label_correct', 'pred_labels', 'pred_max_ious', 'pred_num_detections', 'pred_scores']>


In [85]:
with h5py.File(os.path.join(CHECKPOINT_DIR, 'vahdat_idd_cars_detections_val.hdf5'), 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['frontFar|BLR-2018-04-16_16-14-27_frontFar|0000060', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000006_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000120', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000012_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000180', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000018_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000240', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000024_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000300', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000030_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000360', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000036_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000420', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000042_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000480', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000048_r', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|0000540', 'frontFar|BLR-2018-04-16_16-14-27_frontFar|000054_r', 'frontFar|BLR-2018-04-